In [11]:
%pip install --upgrade -q pip jax jaxlib
%pip install --upgrade -q git+https://github.com/google/flax.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
from typing import Any, Callable, Sequence

import flax
import jax
from flax import linen as nn
from jax import numpy as jnp
from jax import random

## 全結合層

- **Implementation by Flax** : `flax.linen.Dense`
- 「Fead Forward層」、「Prediction層」などと別の名称で呼称されることが多い

### `base`

- 1次元入力に対する全結合層
- $ p[j] = \sum^i n[i] \times w[i, j] + b[j] $
  - $ p $ : 出力
  - $ n $ : 入力
  - $ w $ : 重みパラメータ(入力次元×出力次元)
  - $ b $ : バイアスパラメータ(出力次元)

In [13]:
class Base(nn.Module):
    features: int
    bias_init: Callable = nn.initializers.zeros_init()
    weight_init: Callable = nn.initializers.normal()

    @nn.compact
    def __call__(self, inputs):
        x = inputs
        weight = self.param('weight',
                            self.weight_init,
                            (inputs.shape[-1], self.features))
        y = jnp.dot(x, weight)
        bias = self.param('bias', self.bias_init, (self.features,))
        y = y + bias
        return y

### `Conv1D`

- ２次元ベクトルデータへの畳み込み層
- カーネルサイズが1であるため、全結合層と同様に求めることができる
- $ p[j] = \sum^i n[i] \times w[i, j] + b[j] $
  - $ p $ : 出力
  - $ n $ : 入力
  - $ w $ : 重みパラメータ(入力次元×出力次元)
  - $ b $ : バイアスパラメータ(出力次元)

In [14]:
class Conv1D(nn.Module):
    features: int
    bias_init: Callable = nn.initializers.zeros_init()
    weight_init: Callable = nn.initializers.normal()

    @nn.compact
    def __call__(self, inputs):
        x = inputs
        weight = self.param('weight',
                            self.weight_init,
                            (1, x.shape[-1], self.features))
        batch_size = x.shape[0]
        x_channel = x.shape[-1]

        # カーネルサイズに畳み込み
        x = jnp.reshape(x, (-1, batch_size))
        weight = jnp.reshape(weight, (-1, x_channel))

        y = jnp.dot(x, weight)
        bias = self.param('bias', self.bias_init, (self.features,))
        y = y + bias

        # カーネルを展開
        y = jnp.reshape(y, (batch_size, x.shape[1], self.features))
        return y

## Normalization層

- **Implementaion by Flax** : `flax.linen.LayerNorm`
- $ h_i = f\left(\frac{g_i}{\sigma_i}\left(a_i-\mu_i\right)+b_i \right) $
  - $ \sigma $ : 二乗誤差の平方根
  - $ \mu $ : 平均値
  - $ g $ : 重みパラメータ
  - $ b $ : バイアスパラメータ

In [15]:
class Normalization(nn.Module):
    axis: int = -1
    epsilon: float = 1e-5
    weight_init: Callable = nn.initializers.ones_init()
    bias_init: Callable = nn.initializers.zeros_init()

    @nn.compact
    def __call__(self, input):
        x = input
        # 平均値と二乗誤差
        mean = jnp.mean(x, axis=self.axis, keepdims=True)
        squared_error = jnp.mean(jnp.square(x-mean), axis=self.axis, keepdims=True)
        # 二乗誤差の平方根
        y = (x - mean) * jax.lax.rsqrt(squared_error + self.epsilon)
        weight = self.param('weight', self.weight_init, (input.shape[-1],))
        bias = self.param('bias', self.bias_init, (input.shape[-1]))
        y = y * weight + bias
        return y

## 活性化関数

#### `GELU`

- **Implementation by JAX** : `jax.nn.gelu`

- $ 0.5 * \left( 1 + \tanh\left(\sqrt{2/\pi}\left(x+0.044715 x^3\right)\right) \right) $
- 原点が0を通る
- 負の数では0に近づく
- 正の数では比例する
- 滑らかな連続関数

In [16]:
def gelu(x):
    return 0.5 * (1 + jnp.tanh(jnp.sqrt(2/jnp.pi) * (x + 0.044715 * jnp.pow(x, 3))))

#### `Swish`

- **Implementation by JAX** : `jax.nn.swish`
- $ x \times \text{sigmoid}(x) $
- `GELU`の単純実装

In [17]:
def swish(x):
    return x * jax.nn.sigmoid(x)

#### `Mish`

- **Implementation by JAX** : `jax.nn.mish`
- $ x * \tanh(\text{softplus}(x)) $
- `Swish`よりも性能が良いされている

In [18]:
def mish(x):
    return x * jnp.tanh(jax.nn.softplus(x))

#### `Softmax`

- **Implementation by JAX** : `jax.nn.softmax`
- $ \exp{x_i} / \sum_j \exp{x_j} $
- 全和が1になるように拡大・縮小する

In [21]:
def softmax(x, axis=-1):
    ex = jnp.exp(x - jnp.sum(x, axis=axis, keepdims=True))
    return ex / jnp.sum(ex, axis=axis, keepdims=True)

#### `LogSoftmax`

- **Implementation by JAX** : `jax.nn.log_softmax`
- $ x - \log \left(\sum_j \exp{x_j}\right) $
  - $ \log \left(\exp{x} / \sum{x}\right) $
  - $ = \log\left(\exp{(x)}\right) - \log\left(\sum{\exp(x)}\right)$
  - $ = x - \log\left(\sum{\exp(x)}\right) $

In [1]:
def log_softmax(x, axis=-1):
    ex = jnp.exp(x)
    return x - jnp.log(jnp.sum(ex))

## 損失関数

### Cross Entropy Loss

- **Implementation by Flax** : `optax.losses.safe_softmax_cross_entropy`
- エッジケースによる、極大値や極小値への対応が必要
- Flaxでは損失関数、活性化関数の両方で対応可能

In [2]:
def cross_entropy_loss(labels, logits):
    # batch_size * tokenを一次元ベクトルに展開
    num_vocabulary = logits.shape[-1]
    flatten_labels = jnp.reshape(labels, (-1,))
    flatten_labels = jnp.can_cast(flatten_labels, jnp.int32)
    flatten_logits = jnp.reshape(logits, (-1, num_vocabulary))
    # ラベルをone-hotベクトル化
    one_hot_labels = jax.nn.one_hot(flatten_labels, num_vocabulary, dtype=jnp.float32)
    # クロスエントロピーを算出
    log_probs = log_softmax(flatten_logits)
    # 損失を計算
    loss = -1 * jnp.sum(log_probs * one_hot_labels, axis=(-1))
    # 逆伝播誤差
    loss = jnp.mean(loss)
    return loss


### Masked Cross Entropy Loss

- 